# Assignment Honour Code
I solemnly swear that I have not discussed my assignment solutions with anyone in any way and the solutions I am submitting are my own personal work.

Full Name: Ancy Rex

# QUESTION 1

## Loading the data

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None) 
import requests

# loading the data from data file
data_pf = './data/A2_Q1.csv'
data1 = pd.read_csv(data_pf)
data1=pd.DataFrame(data1)
# new dataset created for working
data1_copy =pd.DataFrame(data1.copy())
data1_copy.head(10)

,ID,Age,Education,Marital_Status,Occupation,Annual_Income
0,1,39,bachelors,never married,professional,high
1,2,50,doctorate,married,professional,mid
2,3,18,high school,never married,agriculture,low
3,4,30,bachelors,married,professional,mid
4,5,37,high school,married,agriculture,mid
5,6,23,high school,never married,agriculture,low
6,7,52,high school,divorced,transport,mid
7,8,40,doctorate,married,professional,high
8,9,46,bachelors,divorced,transport,mid
9,10,33,high school,married,transport,mid


## Part A

Computing the impurity of the column Annual Income which is the target feature in this dataset.

In [2]:
def compute_impurity(feature, impurity_criterion):
    """
    This function calculates impurity of a feature.
    Supported impurity criteria: 'entropy', 'gini'
    input: feature (this needs to be a Pandas series)
    output: feature impurity
    """
    probs = feature.value_counts(normalize=True)
    
    
    if impurity_criterion == 'gini':
        impurity = 1 - np.sum(np.square(probs))
    else:
        raise ValueError('Unknown impurity criterion')
        
    return(round(impurity, 3))

target_entropy = compute_impurity(data1_copy['Annual_Income'], 'gini')
print('Imputrity of target using gini index:',target_entropy)

Imputrity of target using gini index: 0.555


The impurity of the target feature (Annual Income) from the dataset was computed using the gini index. The impurity of Annual Income was found to be 0.55.

## Part B

In this particular dataset the Age column is a continuous column and so it was required to handle it before Gini Index split criterion. 

Age was changed to boolean features by defining threshold and using the threshold partitioned the column using the values of the continuous descriptive feature. The table was sorted using the target feature. The target feature ( Annual Income) has 3 values (low,mid,high).The average of the age where the annual income values change were considered be the threshold values. The age column was then divided to 4 thresholds ( >24, >27 ,>38 ,>42 ). Each threshold has 2 partitions each with set of instances that fall under or over the threshold. The following formulas were used for finding the Partition Gini Index , Remainder and Information Gain.
<center><font size="2"><br    /> Partition Gini Index = 1- $\sum_{low,high,mid}P(AN.INC. = l)$</font></center>
<center><font size="2"><br    />Weight = ($\frac{number of instances for each partition}{total number of instances}$)</font></center>

<center><font size="2"><br  />Remainder = (Weight of first partition * Partition Gini Index of first partition) + (Weight of second partition * Partition Gini Index of second partition)</font></center>

<center><font size="2"><br  />Information Gain = Target impurity - Remainder</font></center>
It was found that >24 has a greater information gain (0.202) compared to the other thresholds. Which makes it the root node in the decision tree.

In [3]:
#age column handled 
age_pf = './data/Age_root.csv'
age = pd.read_csv(age_pf)
age = pd.DataFrame(age)
age = age.replace(np.nan,'',regex=True)
age

,Split by Feature,Partition,Partition Gini Index,Remainder,Information Gain
0,> 24,D1,0.000,0.352,0.202
1,,D2,0.415,,
2,> 27,D3,0.480,0.36,0.195
3,,D4,0.320,,
4,> 38,D5,0.570,0.53,0.025
5,,D6,0.468,,
6,> 42,D7,0.631,0.473,0.081
7,,D8,0.000,,


The function comp_feature_information_gain is a function that consists of calculating the partition impurity, weights of each partition, remainder and information gain .

In [4]:
def comp_feature_information_gain(df, target, descriptive_feature, split_criterion):
    """
    This function calculates information gain for splitting on 
    a particular descriptive feature for a given dataset
    and a given impurity criteria.
    Supported split criterion: 'entropy', 'gini'
    """
    
    print('target feature:', target)
    print('descriptive_feature:', descriptive_feature)
    print('split criterion:', split_criterion)
            
    target_entropy = compute_impurity(data1_copy[target], split_criterion)

    # we define two lists below:
    # entropy_list to store the entropy of each partition
    # weight_list to store the relative number of observations in each partition
    entropy_list = list()
    weight_list = list()
    
    # loop over each level of the descriptive feature
    # to partition the dataset with respect to that level
    # and compute the entropy and the weight of the level's partition
    for level in data1_copy[descriptive_feature].unique():
        data1_copy_feature_level = data1_copy[data1_copy[descriptive_feature] == level]
        entropy_level = compute_impurity(data1_copy_feature_level[target], split_criterion)
        entropy_list.append(round(entropy_level, 3))
        weight_level = len(data1_copy_feature_level) / len(data1_copy)
        weight_list.append(round(weight_level, 3))

    print('impurity of partitions:', entropy_list)
    print('weights of partitions:', weight_list)

    feature_remaining_impurity = np.sum(np.array(entropy_list) * np.array(weight_list))
    print('remaining impurity:', feature_remaining_impurity)
    
    information_gain = target_entropy - feature_remaining_impurity
    print('information gain:', information_gain)
    
    print('====================')

    return(information_gain)

The function above was called to obtain the descriptive feature to use it for gini index criteria.

In [5]:
#gini index criteria
split_criteria = 'gini'
for feature in data1_copy.drop(columns=['Annual_Income','ID','Age']).columns:
    feature_info_gain = comp_feature_information_gain(data1_copy, 'Annual_Income', feature, split_criteria)

target feature: Annual_Income
descriptive_feature: Education
split criterion: gini
impurity of partitions: [0.531, 0.375, 0.625]
weights of partitions: [0.4, 0.2, 0.4]
remaining impurity: 0.5374000000000001
information gain: 0.01759999999999995
target feature: Annual_Income
descriptive_feature: Marital_Status
split criterion: gini
impurity of partitions: [0.611, 0.42, 0.375]
weights of partitions: [0.3, 0.5, 0.2]
remaining impurity: 0.4683
information gain: 0.08670000000000005
target feature: Annual_Income
descriptive_feature: Occupation
split criterion: gini
impurity of partitions: [0.5, 0.5, 0.278]
weights of partitions: [0.4, 0.3, 0.3]
remaining impurity: 0.4334
information gain: 0.12160000000000004


## Part C

Here the descriptive feature Education is considered as the root node and Annual Income as the target. The education feature has values such as high school,bachelors and doctorate. The probability of annual income values in each of the education feature values are found.

The levels in education feature was taken to display the corresponding data partition of education. The probability of values (low,mid,high)in annual incomes where found in each of these values . 
for example: When considering the level bachelors in education. The probability of low,mid and high income was found by the count of(low/mid/high)income/total count of data partition.
This was done for all levels of education

In [6]:
# probability of anuual income values in different levels of education
for level in data1_copy['Education'].unique():
    print('level name:', level)
    df_feature_level = data1_copy[(data1_copy['Education'] == level)]
    print('corresponding data partition:')
    print(df_feature_level)
    df_feature_low = df_feature_level[(data1_copy['Annual_Income'] == 'low')]
    low_pro =(len(df_feature_low))/(len(df_feature_level))
    print('Low probability:',round(low_pro,3))
    df_feature_mid = df_feature_level[(data1_copy['Annual_Income'] == 'mid')]
    mid_pro =(len(df_feature_mid))/(len(df_feature_level))
    print('Mid probability:',round(mid_pro,3))
    df_feature_high = df_feature_level[(data1_copy['Annual_Income'] == 'high')]
    high_pro =(len(df_feature_high))/(len(df_feature_level))
    print('high probability:',round(high_pro,3))
    print('====================')

level name: bachelors
corresponding data partition:
    ID  Age  Education Marital_Status    Occupation Annual_Income
0    1   39  bachelors  never married  professional          high
3    4   30  bachelors        married  professional           mid
8    9   46  bachelors       divorced     transport           mid
12  13   23  bachelors  never married   agriculture           low
14  15   35  bachelors        married   agriculture           mid
15  16   29  bachelors  never married   agriculture           mid
17  18   37  bachelors        married  professional           mid
19  20   25  bachelors        married     transport          high
Low probability: 0.125
Mid probability: 0.625
high probability: 0.25
level name: doctorate
corresponding data partition:
    ID  Age  Education Marital_Status    Occupation Annual_Income
1    2   50  doctorate        married  professional           mid
7    8   40  doctorate        married  professional          high
11  12   45  doctorate        marri

## Resulting Dataframe

df_split was constructed to hold the values of the decriptive features in the dataset. The age columns is divivded into 4 threshold named(Age > 24,Age > 27,Age > 38 and Age > 42) . The remainder and information gain of each feature is displayed. From the dataframe we notice that Age > 24 has the largest information gain with value 0.20 ,thus making it the root node in the decision tree.

In [7]:
# construction of df_split dataframe
df_splits = pd.DataFrame(columns=['Split', 'Remainder', 'Information_Gain','Is_Optimal'])
df_splits.loc[len(df_splits)] = ['Age > 24',0.352,0.202, 'True']
df_splits.loc[len(df_splits)] = ['Age > 27',0.360,0.195, 'False']
df_splits.loc[len(df_splits)] = ['Age > 38',0.530,0.025, 'False']
df_splits.loc[len(df_splits)] = ['Age > 42',0.473,0.081, 'False']
df_splits.loc[len(df_splits)] = ['Education',0.473,0.0175,'False']
df_splits.loc[len(df_splits)] = ['Marital_Status',0.468,0.0867,'False']
df_splits.loc[len(df_splits)] = ['Occupation', 0.433, 0.121,'False']

df_splits.round(3)

,Split,Remainder,Information_Gain,Is_Optimal
0,Age > 24,0.352,0.202,True
1,Age > 27,0.360,0.195,False
2,Age > 38,0.530,0.025,False
3,Age > 42,0.473,0.081,False
4,Education,0.473,0.018,False
5,Marital_Status,0.468,0.087,False
6,Occupation,0.433,0.121,False


df_prediction was constructed to hold the values of the of low , mid and high income probabilities in each level of the education feature. The annual income with the highest probability is put as the leaf_prediction. In this case all the levels in education has the highest mid income probability and thus it would be the leaf_prediction for all levels.


In [8]:
# construction of df_prediction dataframe
df_prediction = pd.DataFrame(columns=['Leaf_Condition', 'Low_Income_Prob', 'Mid_Income_Prob','High_Income_Prob','Leaf_Prediction'])
df_prediction.loc[len(df_prediction)] = ['Education == bachelors',0.125,0.625,0.25,'mid']
df_prediction.loc[len(df_prediction)] = ['Education == doctorate',0.0,0.75,0.25,'mid']
df_prediction.loc[len(df_prediction)] = ['Education == high school', 0.25,0.5,0.25,'mid']


df_prediction

,Leaf_Condition,Low_Income_Prob,Mid_Income_Prob,High_Income_Prob,Leaf_Prediction
0,Education == bachelors,0.125,0.625,0.25,mid
1,Education == doctorate,0.000,0.750,0.25,mid
2,Education == high school,0.250,0.500,0.25,mid
